# 변수를 추출하고 학습데이터, Validation 데이터, Test데이터를 만들어보자

###  MariaDB에서 필요한 데이터를 조회하는 방법

In [1]:
DB_HOST = 'node02'
DB_USER = 'deepbio'
DB_PW = 'deepbio1234'
DB_NAME = 'deepbioDB'

In [2]:
import mysql.connector as mariadb
mariadb_connection = mariadb.connect(host=DB_HOST, user=DB_USER, password=DB_PW, database=DB_NAME)
cursor = mariadb_connection.cursor()
cursor.execute(
    "SELECT id ,disease, platform_type, center, platform, level, barcode, file_name " + 
    "FROM file_manifest " +
    "WHERE file_name like '%rsem.genes.normalized_results' ORDER BY rand()  limit 2")
for ID ,disease, platform_type, center, platform, level, barcode, file_name  in cursor:
    fullPath =  "{}/{}/{}__{}/{}/{}".format(disease, platform_type, center, platform, level, file_name) 
    print fullPath

#select count(*) from file_manifest where file_name like '%rsem.genes.normalized_results';
mariadb_connection.close()

THCA/RNASeqV2/UNC__IlluminaHiSeq_RNASeqV2/Level_3/unc.edu.90c1c52d-0d8b-41d3-82c7-e4773c95a22c.1221217.rsem.genes.normalized_results
COAD/RNASeqV2/UNC__IlluminaHiSeq_RNASeqV2/Level_3/unc.edu.b254c0ed-6844-4298-89a9-d436a0c38ca8.2121736.rsem.genes.normalized_results


### HBASE에서 컨텐츠 데이터 조회하는 방법

In [3]:
import happybase
pool = happybase.ConnectionPool(size=3, host='node03')   

def getHbaseContents( pool, rowkey ) : 
    with pool.connection() as connection:
        table = connection.table('mRNA')
        #print( table )
        #print( table.regions() )
        row = table.row(rowkey)
        return row['data:contents']   # prints the value of cf1:col1

In [ ]:
print getHbaseContents( pool, 'unc.edu.63f7a81e-c404-49fc-bc72-52b2a81ea9a5.1366788.rsem.genes.normalized_results' )

In [ ]:
print getHbaseContents( pool, 'unc.edu.349f670f-1447-4bb0-86ec-fba8734d50ff.2428615.rsem.genes.normalized_results' )

- gene_id 는 GENE NAME | GENE ID
- 예) A1CF|29974 
http://www.ncbi.nlm.nih.gov/gene/29974

### mRNA contents 읽어서 float형 array로 변환함.

In [4]:
contents = '''gene_id	normalized_count
?|100130426	0.0000
?|100133144	1.4054
?|100134869	6.0158
?|10357	168.7013
?|10431	1127.0872
A1BG|1	57.6020
A1CF|29974	0.0000
A2BP1|54715	2.3191
A2LD1|87769	73.1354
''' 

# 20502
#contents = getContents( pool, 'unc.edu.923e19e4-d166-4950-b763-259603ded444.1812289.rsem.genes.normalized_results' ) 
#contents = getContents( pool, 'unc.edu.63f7a81e-c404-49fc-bc72-52b2a81ea9a5.1366788.rsem.genes.normalized_results' ) 

def getGeneVals( contents ) : 
    import StringIO
    buf = StringIO.StringIO( contents )
    line = buf.readline()
    max_id = 0
    count = 0
    gene_vals = []
    while len( line ) > 0 :
        if line.startswith( 'gene_id' ) or line.startswith( '?' ) or line.startswith( '<' ) : 
            line = buf.readline()
            continue 
        
        rows = line.strip().split( '\t' )
        gene = rows[0].split( '|' )
        gene_id = int( gene[1] )
        gene_val = float( rows[1] )
        gene_vals.append( gene_val )
        #print rows[0], rows[1]
        if gene_id  > max_id : max_id = gene_id
        line = buf.readline()
        count += 1
        
    #print max_id, count
    #print len( gene_vals )
    return gene_vals

getGeneVals( contents )

[57.602, 0.0, 2.3191, 73.1354]

### Disease별 데이터수 확인

In [5]:
import mysql.connector as mariadb
mariadb_connection = mariadb.connect(host=DB_HOST, user=DB_USER, password=DB_PW, database=DB_NAME)
cursor = mariadb_connection.cursor()
cursor.execute( 
    "SELECT disease, count(*) " +
    "FROM file_manifest " + 
    "WHERE file_name like '%rsem.genes.normalized_results'  " + 
    "GROUP BY disease ORDER BY disease" )
total_count = 0
for disease, count  in cursor:
    print "{} : {} ".format(disease, count)
    total_count += count

mariadb_connection.close() 

print "TotalCount :{}".format( total_count )

ACC : 79 
BLCA : 433 
BRCA : 1252 
CESC : 309 
CHOL : 45 
COAD : 547 
DLBC : 48 
ESCA : 196 
FPPP : 76 
GBM : 174 
HNSC : 566 
KICH : 91 
KIRC : 614 
KIRP : 323 
LAML : 173 
LGG : 534 
LIHC : 424 
LUAD : 600 
LUSC : 553 
MESO : 87 
OV : 309 
PAAD : 183 
PCPG : 187 
PRAD : 558 
READ : 177 
SARC : 265 
SKCM : 473 
STAD : 450 
TGCT : 156 
THCA : 572 
THYM : 122 
UCEC : 590 
UCS : 57 
UVM : 80 
TotalCount :11303


In [14]:
diseaseDic  = {'ACC'  : 1 
,'BLCA' : 2 
,'BRCA' : 3  
,'CESC' : 4 
,'CHOL' : 5 
,'COAD' : 6 
,'DLBC' : 7 
,'ESCA' : 8 
,'FPPP' : 9 
,'GBM'  : 10
,'HNSC' : 11
,'KICH' : 12
,'KIRC' : 13
,'KIRP' : 14
,'LAML' : 15
,'LGG'  : 16
,'LIHC' : 17
,'LUAD' : 18
,'LUSC' : 19
,'MESO' : 20
,'OV'   : 21
,'PAAD' : 22
,'PCPG' : 23
,'PRAD' : 24
,'READ' : 25
,'SARC' : 26
,'SKCM' : 27
,'STAD' : 28
,'TGCT' : 29
,'THCA' : 30
,'THYM' : 31
,'UCEC' : 32
,'UCS'  : 33
,'UVM'  : 34 }

In [7]:
def datasetFromDisease( disease, diseaseDic, debug=0 ) :

    import mysql.connector as mariadb
    mariadb_connection = mariadb.connect(host=DB_HOST, user=DB_USER, password=DB_PW, database=DB_NAME)
    cursor = mariadb_connection.cursor()
    cursor.execute(
        "SELECT COUNT(*)  FROM file_manifest " +
        "WHERE disease = '{}' AND file_name like '%%rsem.genes.normalized_results' ".format(disease) )
    
    totalCount = cursor.fetchall()[0][0]
    train_set_cnt = totalCount * 70 /  100
    val_set_cnt   = totalCount * 15 /  100
    test_set_cnt  = totalCount * 15 /  100
    
    #print "{}, TotalCount={}, TrainCount={}, ValCount={}, TestCount={}".format(disease, totalCount, train_set_cnt, val_set_cnt, test_set_cnt ) 
    
    train_set_cnt += (totalCount - train_set_cnt - val_set_cnt - test_set_cnt  )
    #print "{}, TotalCount={}, TrainCount={}, ValCount={}, TestCount={}".format(disease, totalCount, train_set_cnt, val_set_cnt, test_set_cnt ) 
    
    cursor.execute(         
        "SELECT disease, file_name  FROM file_manifest " +
        "WHERE disease = '" + disease + "' AND file_name like '%rsem.genes.normalized_results'  " +
        "ORDER BY rand() " )
    
    train_set = []
    val_set = []
    test_set = []
    
    cnt = 0;
    for disease, file_name  in cursor:
        #print "{} : {} ".format(disease, file_name)
        if  train_set_cnt > cnt :
            train_set.append( (file_name, diseaseDic[ disease  ])  )
        elif  val_set_cnt  + train_set_cnt > cnt : 
            val_set.append( (file_name, diseaseDic[ disease  ])  )
        else :
            test_set.append( (file_name, diseaseDic[ disease  ])  )
            
        cnt += 1
    
    if 1 == debug :
        for file_name, y in train_set : print "train : {},  {} ".format(file_name, y)
        for file_name, y in val_set : print "val : {},  {} ".format(file_name, y)
        for file_name, y in test_set : print "test : {},  {} ".format(file_name, y)
      
    mariadb_connection.close() 
    return [train_set, val_set, test_set]

In [ ]:
DATA =  datasetFromDisease( 'BLCA', diseaseDic, 1 )

In [8]:
DATA = datasetFromDisease( 'ACC', diseaseDic )
print  len( DATA[0] ) , len( DATA[1] ) , len( DATA[2] ) 

print DATA[0][0][0]
print DATA[0][0][1]

ACC, TotalCount=79, TrainCount=55, ValCount=11, TestCount=11
ACC, TotalCount=79, TrainCount=57, ValCount=11, TestCount=11
57 11 11
unc.edu.ba350477-3c49-4258-8409-f39447687497.2146316.rsem.genes.normalized_results
1


In [ ]:
#####################################################
# 아래 코드는 학습용 파일이 하나로 생성됨.
# 용량이 너무 크기 나옴.
#####################################################

import time
import gzip, cPickle
import numpy as np

#diseaseDic  = {'ACC'  : 1 , 'BLCA' : 2   }

diseases = diseaseDic.keys()
train_set_x = []
train_set_y = []
val_set_x = []
val_set_y = []
test_set_x = []
val_set_y = []

now = time.localtime()
s = "%04d-%02d-%02d_%02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
filename = '/home/deepbio/note/work/tcga/data/mrna_{}.pkl.gz'.format( s)
print filename
f = gzip.open(filename ,'wb')
    
for disease in diseases : 
    print "disease={}".format(disease)
    data = datasetFromDisease( disease, diseaseDic )
    train_data = data[0]
    for train  in train_data : 
        contents = getHbaseContents( pool, train[0]  )
        train_set_x.append( getGeneVals( contents ) )
        train_set_y.append( train[1] ) 
    
    val_data = data[1]
    for val  in train_data : 
        contents = getHbaseContents( pool, val[0]  )
        val_set_x.append( getGeneVals( contents ) )
        val_set_y.append( val[1] ) 
                
    test_data = data[2]
    for test  in test_data : 
        contents = getHbaseContents( pool, test[0]  )
        test_set_x.append( getGeneVals( contents )  )
        val_set_y.append( test[1] ) 

    train_set = np.array(train_set_x), np.array(train_set_y)
    val_set   = np.array(val_set_x),  np.array(val_set_y)
    test_set  = np.array(test_set_x),  np.array(val_set_y)
                     
    dataset = [train_set, val_set, test_set]
                     
    cPickle.dump(dataset, f, protocol=2)
    del  dataset 
    
f.close()

In [9]:
import mysql.connector as mariadb
mariadb_connection = mariadb.connect(host=DB_HOST, user=DB_USER, password=DB_PW, database=DB_NAME)

def insertTrainData(dbconnection , data_set, type = 1) : 
    # type =1 은 train, type=2 는 validation, type=3은 test
    cursor = dbconnection.cursor()
    
    for data  in data_set : 
        sql = "INSERT INTO train_manifest (file_name, y_val, type ) VALUES ( '{}', {} ,{}  )".format(  data[0], data[1], type )
        #print sql
        try:
            if  None !=  cursor and  None != sql  :  cursor.execute( sql )
        except mariadb.Error as error:
            print("Error: {}".format(error))

    dbconnection.commit() 
    
mariadb_connection.close() 

In [13]:
import time
import gzip, cPickle
import numpy as np

import mysql.connector as mariadb
mariadb_connection = mariadb.connect(host=DB_HOST, user=DB_USER, password=DB_PW, database=DB_NAME)

def saveDataSet(dbconnection, basename, type=1) :
    cursor = mariadb_connection.cursor()

    cursor.execute("SELECT count(*) as totalcount FROM train_manifest WHERE  type = %d " % type )
    totalCount = cursor.fetchone() 
    print "totalCount=", totalCount    

    offset = 0;
    QUERY_FETCH_SIZE = 500
    pageCount = totalCount[0] / QUERY_FETCH_SIZE 
    if totalCount[0] % QUERY_FETCH_SIZE  > 0 : 
        pageCount += 1

    train_set_x = []
    train_set_y = []
    for idx in range( pageCount )  :
        offset = idx * QUERY_FETCH_SIZE
        cursor.execute("SELECT file_name, y_val, type, id type FROM train_manifest WHERE type = %d  ORDER BY file_name  LIMIT %d, %d" %(type, offset,QUERY_FETCH_SIZE))
        for file_name, y_val, types, ids  in cursor:
            #fullPath =  "{}_{}_{}_{}".format(file_name, y_val, types, ids) 
            #print fullPath
            contents = getHbaseContents( pool,file_name  )
            train_set_x.append( getGeneVals( contents ) )
            train_set_y.append( y_val ) 
            
        train_set = np.array(train_set_x), np.array(train_set_y)            
        filename = basename + ( "_type%d_%02d.pkl.gz"%(type, idx) )
        print filename
        f = gzip.open(filename ,'wb')
        cPickle.dump(train_set, f, protocol=2)
        f.close()
        
        train_set_x = []
        train_set_y = []
            
#########################
## 테스트용 코드
now = time.localtime()
s = "%04d%02d%02d-%02d%02d%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
basename = '/home/deepbio/note/work/tcga/data/mrna_{}'.format( s)

#saveDataSet(mariadb_connection, basename, 1)
mariadb_connection.close() 
#########################

In [15]:
#diseaseDic  = {'ACC'  : 1 , 'BLCA' : 2   }

import mysql.connector as mariadb
mariadb_connection = mariadb.connect(host=DB_HOST, user=DB_USER, password=DB_PW, database=DB_NAME)

cursor = mariadb_connection.cursor()
cursor.execute( "DELETE FROM train_manifest WHERE id > 0 " )
mariadb_connection.commit() 

diseases = diseaseDic.keys()
for disease in diseases : 
    print "disease={}".format(disease)
    data = datasetFromDisease( disease, diseaseDic )
    train_data = data[0]
    val_data = data[1]
    test_data = data[2]
    insertTrainData(mariadb_connection,  train_data, 1 )
    insertTrainData(mariadb_connection,  val_data, 2 )
    insertTrainData(mariadb_connection,  test_data, 3 )


now = time.localtime()
s = "%04d%02d%02d-%02d%02d%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
basename = '/home/deepbio/note/work/tcga/data/mrna_{}'.format( s )

saveDataSet(mariadb_connection, basename, 1)
saveDataSet(mariadb_connection, basename, 2)
saveDataSet(mariadb_connection, basename, 3)
    
mariadb_connection.close() 

disease=ACC
ACC, TotalCount=79, TrainCount=55, ValCount=11, TestCount=11
ACC, TotalCount=79, TrainCount=57, ValCount=11, TestCount=11
disease=DLBC
DLBC, TotalCount=48, TrainCount=33, ValCount=7, TestCount=7
DLBC, TotalCount=48, TrainCount=34, ValCount=7, TestCount=7
disease=PAAD
PAAD, TotalCount=183, TrainCount=128, ValCount=27, TestCount=27
PAAD, TotalCount=183, TrainCount=129, ValCount=27, TestCount=27
disease=TGCT
TGCT, TotalCount=156, TrainCount=109, ValCount=23, TestCount=23
TGCT, TotalCount=156, TrainCount=110, ValCount=23, TestCount=23
disease=LUSC
LUSC, TotalCount=553, TrainCount=387, ValCount=82, TestCount=82
LUSC, TotalCount=553, TrainCount=389, ValCount=82, TestCount=82
disease=READ
READ, TotalCount=177, TrainCount=123, ValCount=26, TestCount=26
READ, TotalCount=177, TrainCount=125, ValCount=26, TestCount=26
disease=GBM
GBM, TotalCount=174, TrainCount=121, ValCount=26, TestCount=26
GBM, TotalCount=174, TrainCount=122, ValCount=26, TestCount=26
disease=THCA
THCA, TotalCount=5

In [16]:
! ls -al  /home/deepbio/note/work/tcga/data/

total 707776
drwxrwxr-x 3 deepbio deepbio     4096 Jan 25 22:23 .
drwxrwxr-x 4 deepbio deepbio     4096 Jan 25 09:48 ..
drwxrwxr-x 2 deepbio deepbio     4096 Jan 25 18:27 bak
-rw-rw-r-- 1 deepbio deepbio 32216845 Jan 25 20:13 mrna_20160125-200855_type1_00.pkl.gz
-rw-rw-r-- 1 deepbio deepbio 32096294 Jan 25 20:19 mrna_20160125-200855_type1_01.pkl.gz
-rw-rw-r-- 1 deepbio deepbio 32032107 Jan 25 20:24 mrna_20160125-200855_type1_02.pkl.gz
-rw-rw-r-- 1 deepbio deepbio 32024240 Jan 25 20:30 mrna_20160125-200855_type1_03.pkl.gz
-rw-rw-r-- 1 deepbio deepbio 31956428 Jan 25 20:35 mrna_20160125-200855_type1_04.pkl.gz
-rw-rw-r-- 1 deepbio deepbio 32039357 Jan 25 20:42 mrna_20160125-200855_type1_05.pkl.gz
-rw-rw-r-- 1 deepbio deepbio 32070663 Jan 25 20:48 mrna_20160125-200855_type1_06.pkl.gz
-rw-rw-r-- 1 deepbio deepbio 32204918 Jan 25 20:54 mrna_20160125-200855_type1_07.pkl.gz
-rw-rw-r-- 1 deepbio deepbio 32198752 Jan 25 20:59 mrna_20160125-200855_type1_08.pkl.gz
-rw-rw-r-- 1 deepbio deepbio 3217

http://stackoverflow.com/questions/26107927/how-to-put-my-dataset-in-a-pkl-file-in-the-exact-format-and-data-structure-used

```
from PIL import Image
from numpy import genfromtxt
import gzip, cPickle
from glob import glob
import numpy as np
import pandas as pd
Data, y = dir_to_dataset("trainMNISTForm\\*.BMP","trainLabels.csv")
# Data and labels are read 

train_set_x = Data[:2093]
val_set_x = Data[2094:4187]
test_set_x = Data[4188:6281]
train_set_y = y[:2093]
val_set_y = y[2094:4187]
test_set_y = y[4188:6281]
# Divided dataset into 3 parts. I had 6281 images.

train_set = train_set_x, train_set_y
val_set = val_set_x, val_set_y
test_set = test_set_x, val_set_y

dataset = [train_set, val_set, test_set]

f = gzip.open('file.pkl.gz','wb')
cPickle.dump(dataset, f, protocol=2)
f.close()

```